In [1]:
import networkx as nx
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
import numpy as np
import math
import pickle
from tqdm import tqdm_notebook
from multiprocessing import Pool

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.optim.lr_scheduler import ExponentialLR

In [3]:
torch.cuda.set_device(0)

## Model Declaration

In [4]:
class GRU(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(GRU, self).__init__()
        
        self.in_dim = in_dim
        self.hid1 = 200
        self.hid2 = 100
        self.rnn = nn.GRU(self.in_dim, self.hid1, num_layers=3, 
                          bidirectional=True, batch_first=True, dropout=0.3)
        self.relu = nn.ReLU()
        
        self.out1 = nn.Sequential(
            nn.Linear(2*self.hid1, 64),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            
            nn.Linear(32, out_dim),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        rnn, hid = self.rnn(x)
        return self.out1(self.relu(rnn[:, -1]))
        
    def get_trainable_parameters(self):
        return (param for param in self.parameters() if param.requires_grad)

In [5]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.01, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce
    
    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        
        if self.reduce:
            return torch.mean(F_loss)
        else:
            F_loss

In [6]:
class FocalLoss2(nn.Module):
    def __init__(self, alpha=0.01, gamma_pos=3, gamma_neg=2, logits=False, reduce=True):
        super(FocalLoss2, self).__init__()
        self.alpha = alpha
        self.gamma_pos = gamma_pos
        self.gamma_neg = gamma_neg
        self.logits = logits
        self.reduce = reduce
    
    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        gamma_diff = self.gamma_pos - self.gamma_neg
        F_loss_pos = self.alpha * targets * (1-pt)**self.gamma_pos * BCE_loss
        F_loss_pos = torch.mean(pt)**(-gamma_diff) * F_loss_pos
        F_loss_neg = self.alpha * (1 - targets) * (1-pt)**self.gamma_neg * BCE_loss
        F_loss = F_loss_pos + F_loss_neg
        
        avg_F_loss_pos = torch.sum(F_loss_pos) / torch.sum(targets)
        avg_F_loss_neg = torch.sum(F_loss_neg) / torch.sum(1-targets)
        
        if self.reduce:
            return torch.mean(F_loss), avg_F_loss_pos, avg_F_loss_neg
        else:
            return F_loss, F_loss_pos, F_loss_neg

## Virtual Adversarial Training

In [7]:
import contextlib


@contextlib.contextmanager
def _disable_tracking_bn_stats(model):

    def switch_attr(m):
        if hasattr(m, 'track_running_stats'):
            m.track_running_stats ^= True
            
    model.apply(switch_attr)
    yield
    model.apply(switch_attr)

    
def _l2_normalize(d):
    d_reshaped = d.view(d.shape[0], -1, *(1 for _ in range(d.dim() - 2)))
    d /= torch.norm(d_reshaped, dim=1, keepdim=True) + 1e-8
    return d


class VATLoss(nn.Module):

    def __init__(self, xi=1e-6, eps=0.1, ip=1):
        """VAT loss
        :param xi: hyperparameter of VAT (default: 10.0)
        :param eps: hyperparameter of VAT (default: 1.0)
        :param ip: iteration times of computing adv noise (default: 1)
        """
        super(VATLoss, self).__init__()
        self.xi = xi
        self.eps = eps
        self.ip = ip

    def forward(self, model, x):
        with torch.no_grad():
            pred = F.softmax(model(x), dim=1)

        # prepare random unit tensor
        d = torch.rand(x.shape).sub(0.5).to(x.device)
        d = _l2_normalize(d)

        with _disable_tracking_bn_stats(model):
            # calc adversarial direction
            for _ in range(self.ip):
                d.requires_grad_()
                pred_hat = model(x + self.xi * d)
                logp_hat = F.log_softmax(pred_hat, dim=1)
                #adv_distance = F.kl_div(logp_hat, pred, reduction='batchmean')   # for PyTorch v1.0
                adv_distance = F.kl_div(logp_hat, pred)         # for PyTorch v0.4
                adv_distance.backward()
                d = _l2_normalize(d.grad)
                model.zero_grad()
    
            # calc LDS
            r_adv = d * self.eps
            pred_hat = model(x + r_adv)
            logp_hat = F.log_softmax(pred_hat, dim=1)
            #lds = F.kl_div(logp_hat, pred, reduction='batchmean')    # for PyTorch v1.0
            lds = F.kl_div(logp_hat, pred)          # for PyTorch v1.0

        return lds


class VATLoss2(nn.Module):

    def __init__(self, xi=1e-6, eps_pos=100, eps_neg=1., ip=1):
        """VAT loss
        :param xi: hyperparameter of VAT (default: 10.0)
        :param eps_pos: hyperparameter of VAT (default: 100.0)
        :param eps_neg: hyperparameter of VAT (default: 0.1)
        :param ip: iteration times of computing adv noise (default: 1)
        """
        super(VATLoss2, self).__init__()
        self.xi = xi
        self.eps_pos = eps_pos
        self.eps_neg = eps_neg
        self.ip = ip

    def forward(self, model, x, y):
        with torch.no_grad():
            pred = F.softmax(model(x), dim=1)

        # prepare random unit tensor
        d = torch.rand(x.shape).sub(0.5).to(x.device)
        d = _l2_normalize(d)

        with _disable_tracking_bn_stats(model):
            # calc adversarial direction
            for _ in range(self.ip):
                d.requires_grad_()
                pred_hat = model(x + self.xi * d)
                logp_hat = F.log_softmax(pred_hat, dim=1)
                #adv_distance = F.kl_div(logp_hat, pred, reduction='batchmean')   # for PyTorch v1.0
                adv_distance = F.kl_div(logp_hat, pred)         # for PyTorch v0.4
                adv_distance.backward()
                d = _l2_normalize(d.grad)
                model.zero_grad()
    
            # calc LDS
            r_adv = d * (self.eps_pos * y + self.eps_neg * (1-y)).reshape(-1, 1, 1)
            pred_hat = model(x + r_adv)
            logp_hat = F.log_softmax(pred_hat, dim=1)
            #lds = F.kl_div(logp_hat, pred, reduction='batchmean')    # for PyTorch v1.0
            lds = F.kl_div(logp_hat, pred)          # for PyTorch v1.0

        return lds

## Parameters Settings

In [8]:
#
# Classifier
# ---------------------
## focal loss
alpha = 1
gamma_pos = 6
gamma_neg = 2
learn_rate = 1e-5
grad_clip = 1

#
# VAT
# ---------------------
vat_xi = 1e-6
vat_eps_pos = 1
vat_eps_neg = 0.01
vat_ip = 1

#
# Training process
# ---------------------
train_batch_size = 128
test_batch_size = 256

max_epochs = 100

## Data Preparation

In [9]:
train_data = np.load('datasets/Training_data_heter.npz', allow_pickle=True)
test_data  = np.load('datasets/Testing_data_heter.npz',  allow_pickle=True)

training_data, training_label, training_announce, training_FILTER = train_data['arr_0'], train_data['arr_1'], train_data['arr_2'], train_data['arr_3']
testing_data,  testing_label,  testing_announce,  testing_FILTER = test_data['arr_0'], test_data['arr_1'], test_data['arr_2'], test_data['arr_3']

X_train = training_data#[(training_announce == 1) & (training_FILTER == 0 )]
y_train = training_label#[(training_announce == 1) & (training_FILTER == 0 )]

X_test  = testing_data#[(testing_announce == 1) & (testing_FILTER == 0 )]
y_test  = testing_label#[(testing_announce == 1) & (testing_FILTER == 0 )]

In [10]:
# data = np.load('GRUArray_and_label_for_NewEmbedding_heter_superv_recur_focal_logisticMF.npz', allow_pickle=True)

# GPUArray = data['arr_0']
# label = data['arr_1']

# GPUArray = GPUArray[-1033905:,:,:]
# label = label[-1033905:]

# X_train, X_test, y_train, y_test = train_test_split(GPUArray, label, random_state=42)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

train_data = []
for i in range(len(X_train)):
    train_data.append((X_train[i], y_train[i]))
    
test_data = []
for i in range(len(X_test)):
    test_data.append((X_test[i], y_test[i]))

train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=test_batch_size)

In [11]:
classifier = GRU(in_dim=X_train.shape[2], out_dim=2).cuda()
focal_loss = FocalLoss2(alpha, gamma_pos, gamma_neg)
optim_clsfr = optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()), 
                         lr=learn_rate)

In [12]:
vat_loss2 = VATLoss2(xi=vat_xi, eps_pos=vat_eps_pos, eps_neg=vat_eps_neg, ip=vat_ip)

In [13]:
def train(epoch, dataloader, clip_grad_norm=0):
    label_list = []
    pred_y_list = []
    
    clsf_loss_batch = []
    clsf_loss_pos_batch = []
    clsf_loss_neg_batch = []
    vat_batch = []
    for batch_idx, (data, target) in enumerate(dataloader):
        if data.size()[0] != dataloader.batch_size:
            continue
        data, target = Variable(data.cuda()), Variable(target.cuda())
        tmp = target.reshape(-1, 1)
        onehot_target = torch.cat([1-tmp, tmp], dim=1)
        
        #
        # Update classifier on real samples
        #
        optim_clsfr.zero_grad()
        
        vat_kld = vat_loss2(classifier, data, target)
        
        pred_y = classifier(data).squeeze(-1)
        
        clsf_loss, clsf_loss_pos, clsf_loss_neg = focal_loss(pred_y, onehot_target)
        loss = clsf_loss + vat_kld
        
        loss.backward()
        
        # gradient clipping
        if clip_grad_norm > 0:
            torch.nn.utils.clip_grad_norm_(classifier.parameters(), max_norm=clip_grad_norm)
        
        optim_clsfr.step()
        
        #
        # Record the losses
        #
        pred_yy = torch.nn.functional.softmax(pred_y, dim=1)[:, 1]
        vat_batch.append(vat_kld)
        clsf_loss_batch.append(clsf_loss)
        if torch.sum(target) > 0:
            clsf_loss_pos_batch.append(clsf_loss_pos)
        clsf_loss_neg_batch.append(clsf_loss_neg)
        
        label_list += list(target.cpu().detach().numpy())
        pred_y_list += list(pred_yy.cpu().detach().numpy())
        
        #if batch_idx % 2000 == 0:
        #    print('  Idx {} => clsf: {}'.format(batch_idx, clsf_loss))
    
    vat_loss_avg = sum(vat_batch) / len(vat_batch)
    clsf_loss_avg = sum(clsf_loss_batch) / len(clsf_loss_batch)
    clsf_loss_pos_avg = sum(clsf_loss_pos_batch) / len(clsf_loss_pos_batch)
    clsf_loss_neg_avg = sum(clsf_loss_neg_batch) / len(clsf_loss_neg_batch)
    
    return np.array(label_list), np.array(pred_y_list), clsf_loss_avg, clsf_loss_pos_avg, clsf_loss_neg_avg, vat_loss_avg

In [14]:
def infer(dataloader):
    label_list = []
    pred_y_list = []   
    
    clsf_loss_batch = []
    clsf_loss_pos_batch = []
    clsf_loss_neg_batch = []
    for batch_idx, (data, target) in enumerate(dataloader):
        if data.size()[0] != dataloader.batch_size:
            continue
        data, target = Variable(data.cuda()), Variable(target.cuda())
         
        # Update classifier
        
        pred_y = classifier(data).squeeze(-1)
        pred_y = torch.nn.functional.softmax(pred_y, dim=1)[:, 1]
        
        clsf_loss, clsf_loss_pos, clsf_loss_neg = focal_loss(pred_y, target)
        clsf_loss_batch.append(clsf_loss)
        if torch.sum(target) > 0:
            clsf_loss_pos_batch.append(clsf_loss_pos)
        clsf_loss_neg_batch.append(clsf_loss_neg)
        
        label_list += list(target.cpu().detach().numpy())
        pred_y_list += list(pred_y.cpu().detach().numpy())
    
    clsf_loss_avg = sum(clsf_loss_batch) / len(clsf_loss_batch)
    clsf_loss_pos_avg = sum(clsf_loss_pos_batch) / len(clsf_loss_pos_batch)
    clsf_loss_neg_avg = sum(clsf_loss_neg_batch) / len(clsf_loss_neg_batch)
    
    return np.array(label_list), np.array(pred_y_list), clsf_loss_avg, clsf_loss_pos_avg, clsf_loss_neg_avg

In [15]:
def evaluate(y_true, y_pred):
    prec = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return prec, recall, f1

In [16]:
train_history_loss = []
train_history_auc = []
max_thres = 0.
max_train_auc = 0.


print('Parameter Setting ----------------------------------------------------------------------')
print('Model = VAT + GRU')
print('conv1d use activation = {}'.format(True))
print('graph_emdeding = Heter_AutoEncoder')
print('alpha = {}'.format(alpha))
print('gamma_pos = {}'.format(gamma_pos))
print('gamma_neg = {}'.format(gamma_neg))
print('learn_rate = {}'.format(learn_rate))
print('train_batch_size = {}'.format(train_batch_size))
print('test_batch_size = {}'.format(test_batch_size))
print('max_epochs = {}'.format(max_epochs))
print('vat_xi = {}'.format(vat_xi))
print('vat_eps_pos = {}'.format(vat_eps_pos))
print('vat_eps_neg = {}'.format(vat_eps_neg))
print('vat_ip = {}'.format(vat_ip))

print('\n')


for epoch in range(max_epochs):
    print('Epoch {} -------------------------------------------------------------------------'.format(epoch))
    
    classifier.train()
    label_train, pred_y_train, clsf_loss_train, clsf_loss_pos_train, \
        clsf_loss_neg_train, vat_loss_train = train(epoch, train_dataloader, clip_grad_norm=1)
    
    auc_train = roc_auc_score(label_train, pred_y_train)
    train_history_loss.append(clsf_loss_train)
    train_history_auc.append(auc_train)
    print('    Training => auc: {:.6f}, clsf_pos: {}, clsf_neg: {}, vat_loss: {}'.
          format(auc_train, clsf_loss_pos_train, clsf_loss_neg_train, vat_loss_train))
    thres = np.min(pred_y_train[label_train==1]) - 1e-6
    print("                Threshold is set to {}".format(thres))
    y_predict_bin = np.array(pred_y_train > thres, dtype=int)
    prec_train, recall_train, f1_train = evaluate(label_train, y_predict_bin)
    print('                prec: {:.4f}, rec: {:.4f}, F1score: {:.4f}'.
          format(prec_train, recall_train, f1_train))
    
    if epoch % 1 == 0:
        #
        # Testing
        # ------------------------------------------------------------------------------------        
        with torch.no_grad():
            classifier.eval()
            label_test, pred_y_test, clsf_loss_test, clsf_loss_pos_test, clsf_loss_neg_test = infer(test_dataloader)    
        
        auc = roc_auc_score(label_test, pred_y_test)
        
        print("            Min. Probailities on test set with label 1: {}".
              format(np.min(pred_y_test[label_test==1])))
        y_predict_bin = np.array(pred_y_test > thres, dtype=int)
        prec, recall, f1 = evaluate(label_test, y_predict_bin)
                
        print('    Testing ==> auc: {:.6f}, prec: {:.4f}, rec: {:.4f}, F1score: {:.4f}, clsf_loss: {}'.
              format(auc, prec, recall, f1, clsf_loss_test))
        
        if auc_train > max_train_auc:
            max_train_auc = auc_train if auc_train > max_train_auc else max_train_auc
            torch.save({'epoch': epoch,
                        'model_state_dict': classifier.state_dict(),
                        'optimizer_state_dict': optim_clsfr.state_dict(),
                        'loss': focal_loss,
                       }, 
                       'saved_models/VATConv_heter_clsfr_xi{}_eps{}{}_focal{}{}_BestAUC'.
                       format(int(-math.log10(vat_xi)), int(-math.log10(vat_eps_pos)), 
                              int(-math.log10(vat_eps_neg)), gamma_pos, gamma_neg))
#         if thres > max_thres:
#             max_thres = thres if thres > max_thres else max_thres
#             torch.save({'epoch': epoch,
#                         'model_state_dict': classifier.state_dict(),
#                         'optimizer_state_dict': optim_clsfr.state_dict(),
#                         'loss': focal_loss,
#                        }, 
#                        'saved_models/VATConv_heter_clsfr_xi{}_eps{}{}_focal{}{}_BestThres'.
#                        format(int(-math.log10(vat_xi)), int(-math.log10(vat_eps_pos)), 
#                               int(-math.log10(vat_eps_neg)), gamma_pos, gamma_neg))
        
        

Parameter Setting ----------------------------------------------------------------------
Model = VAT + GRU
conv1d use activation = True
graph_emdeding = Heter_AutoEncoder
alpha = 1
gamma_pos = 6
gamma_neg = 2
learn_rate = 1e-05
train_batch_size = 128
test_batch_size = 256
max_epochs = 100
vat_xi = 1e-06
vat_eps_pos = 1
vat_eps_neg = 0.01
vat_ip = 1


Epoch 0 -------------------------------------------------------------------------
    Training => auc: 0.579337, clsf_pos: 0.02913891337811947, clsf_neg: 0.01415943168103695, vat_loss: 0.000749719503801316
                Threshold is set to 0.28217197792434695
                prec: 0.0003, rec: 1.0000, F1score: 0.0005
            Min. Probailities on test set with label 1: 0.28631317615509033
    Testing ==> auc: 0.962559, prec: 0.0002, rec: 1.0000, F1score: 0.0004, clsf_loss: 0.0270131416618824
Epoch 1 -------------------------------------------------------------------------
    Training => auc: 0.913817, clsf_pos: 0.014482252299785614, 

                prec: 0.2092, rec: 1.0000, F1score: 0.3460
            Min. Probailities on test set with label 1: 0.26897916197776794
    Testing ==> auc: 0.999857, prec: 0.0971, rec: 0.9565, F1score: 0.1764, clsf_loss: 0.022691603749990463
Epoch 16 -------------------------------------------------------------------------
    Training => auc: 0.999911, clsf_pos: 0.000582128472160548, clsf_neg: 4.264128801878542e-05, vat_loss: 1.7003708308038767e-06
                Threshold is set to 0.33063851134681704
                prec: 0.2802, rec: 1.0000, F1score: 0.4378
            Min. Probailities on test set with label 1: 0.26896026730537415
    Testing ==> auc: 0.999858, prec: 0.3964, rec: 0.9565, F1score: 0.5605, clsf_loss: 0.02268538996577263
Epoch 17 -------------------------------------------------------------------------
    Training => auc: 0.999925, clsf_pos: 0.0006413144874386489, clsf_neg: 4.242271097609773e-05, vat_loss: 1.616114673197444e-06
                Threshold is set to 0

            Min. Probailities on test set with label 1: 0.2691650092601776
    Testing ==> auc: 0.999842, prec: 0.3014, rec: 0.9565, F1score: 0.4583, clsf_loss: 0.022671157494187355
Epoch 32 -------------------------------------------------------------------------
    Training => auc: 0.999975, clsf_pos: 0.0006224146927706897, clsf_neg: 2.0643239622586407e-05, vat_loss: 1.2847949619754218e-06
                Threshold is set to 0.2725057436695099
                prec: 0.0808, rec: 1.0000, F1score: 0.1495
            Min. Probailities on test set with label 1: 0.26916739344596863
    Testing ==> auc: 0.999841, prec: 0.0773, rec: 0.9565, F1score: 0.1431, clsf_loss: 0.02266937494277954
Epoch 33 -------------------------------------------------------------------------
    Training => auc: 0.999996, clsf_pos: 0.00033516937401145697, clsf_neg: 1.2673761375481263e-05, vat_loss: 1.0666698244676809e-06
                Threshold is set to 0.2904922200908661
                prec: 0.3746, rec: 1.0

    Testing ==> auc: 0.999846, prec: 0.1071, rec: 0.9565, F1score: 0.1926, clsf_loss: 0.02267390489578247
Epoch 48 -------------------------------------------------------------------------
    Training => auc: 0.999980, clsf_pos: 0.0003530214016791433, clsf_neg: 1.0725836546043865e-05, vat_loss: 2.480497641954571e-06
                Threshold is set to 0.2724270655384064
                prec: 0.0928, rec: 1.0000, F1score: 0.1699
            Min. Probailities on test set with label 1: 0.2697741389274597
    Testing ==> auc: 0.999865, prec: 0.0398, rec: 0.9565, F1score: 0.0764, clsf_loss: 0.022670121863484383
Epoch 49 -------------------------------------------------------------------------
    Training => auc: 0.999986, clsf_pos: 0.0003138768079224974, clsf_neg: 9.253452844859567e-06, vat_loss: 2.1107146039867075e-06
                Threshold is set to 0.276205374168396
                prec: 0.1328, rec: 1.0000, F1score: 0.2344
            Min. Probailities on test set with label 1: 0.2

    Training => auc: 0.999991, clsf_pos: 0.00030160462483763695, clsf_neg: 1.0148997716896702e-05, vat_loss: 1.2563561995193595e-06
                Threshold is set to 0.27386106388473513
                prec: 0.1940, rec: 1.0000, F1score: 0.3249
            Min. Probailities on test set with label 1: 0.27013930678367615
    Testing ==> auc: 0.999852, prec: 0.0446, rec: 0.9783, F1score: 0.0852, clsf_loss: 0.022675495594739914
Epoch 65 -------------------------------------------------------------------------
    Training => auc: 0.999993, clsf_pos: 0.0002665621286723763, clsf_neg: 8.151188922056463e-06, vat_loss: 1.5302942983907997e-06
                Threshold is set to 0.27726422088050845
                prec: 0.2176, rec: 1.0000, F1score: 0.3574
            Min. Probailities on test set with label 1: 0.2695048749446869
    Testing ==> auc: 0.999854, prec: 0.0815, rec: 0.9565, F1score: 0.1502, clsf_loss: 0.022669723257422447
Epoch 66 ---------------------------------------------------

                prec: 0.0972, rec: 1.0000, F1score: 0.1772
            Min. Probailities on test set with label 1: 0.2691144049167633
    Testing ==> auc: 0.999866, prec: 0.0484, rec: 0.9783, F1score: 0.0923, clsf_loss: 0.0226663276553154
Epoch 81 -------------------------------------------------------------------------
    Training => auc: 0.999998, clsf_pos: 0.00015393458306789398, clsf_neg: 6.272867722145747e-06, vat_loss: 1.2552768566820305e-06
                Threshold is set to 0.2896837724914551
                prec: 0.5396, rec: 1.0000, F1score: 0.7010
            Min. Probailities on test set with label 1: 0.26901277899742126
    Testing ==> auc: 0.999873, prec: 0.9318, rec: 0.8913, F1score: 0.9111, clsf_loss: 0.022665148600935936
Epoch 82 -------------------------------------------------------------------------
    Training => auc: 0.999988, clsf_pos: 0.0003733557532541454, clsf_neg: 1.0006689990404993e-05, vat_loss: 1.0039104836323531e-06
                Threshold is set to 

            Min. Probailities on test set with label 1: 0.2690748870372772
    Testing ==> auc: 0.999869, prec: 0.1034, rec: 0.9348, F1score: 0.1861, clsf_loss: 0.022665325552225113
Epoch 97 -------------------------------------------------------------------------
    Training => auc: 0.999993, clsf_pos: 0.000361113139661029, clsf_neg: 8.202393473766278e-06, vat_loss: 1.0001004966397886e-06
                Threshold is set to 0.27466122643852237
                prec: 0.2329, rec: 1.0000, F1score: 0.3778
            Min. Probailities on test set with label 1: 0.26905950903892517
    Testing ==> auc: 0.999865, prec: 0.1439, rec: 0.8913, F1score: 0.2477, clsf_loss: 0.022665316238999367
Epoch 98 -------------------------------------------------------------------------
    Training => auc: 0.999986, clsf_pos: 0.0004973529721610248, clsf_neg: 1.1930634173040744e-05, vat_loss: 8.471837418255745e-07
                Threshold is set to 0.26987342374229434
                prec: 0.1256, rec: 1.00